### Основные команды

In [ ]:
import requests
from bs4 import BeautifulSoup

# Шаг 1: Получаем HTML-код страницы
url = 'https://parsinger.ru/4.3/4/index.html'
response = requests.get(url)
html_content = response.content

# Шаг 2: Парсим HTML-код
soup = BeautifulSoup(html_content, 'html.parser')

# Шаг 3: Ищем все теги p
p_tags = soup.find_all('p')

# Шаг 4: Извлекаем значение атрибутов id и class
total_id_sum = 0
total_class_sum = 0
for p in p_tags:
    if len(p.text.replace(' ', '')) % 2 == 0: # стираем пробелы и проверяем длину строки на четность
        total_id_sum += int(p.get('id'))
        total_class_sum += int(p.get('class')[0])
print(total_id_sum + total_class_sum)

### Все возможные поля в ответе response.

In [ ]:
# URL для примеров
url = "https://httpbin.org/user-agent"
# Выполняем GET-запрос
    response = requests.get(url)
# status_code: HTTP-код статуса ответа.
    print("HTTP-код статуса ответа:", response.status_code)
# text: Текстовое представление содержимого ответа.
    print("Текстовое содержимое ответа:", response.text)
# content: Содержимое ответа в виде байтов.
    print("Содержимое ответа в виде байтов:", response.content)
# json: Метод для десериализации JSON-ответа.
    json_response = response.json()
    print("Десериализованный JSON-ответ:", json_response)
# headers: Заголовки HTTP, возвращаемые сервером.
    print("Заголовки HTTP:", response.headers)
# url: Исходный URL-адрес, на который был выполнен запрос.
    print("Исходный URL-адрес запроса:", response.url)
# encoding: Кодировка ответа.
    print("Кодировка ответа:", response.encoding)
# elapsed: Время, затраченное на выполнение запроса.
    print("Время выполнения запроса:", response.elapsed)
# cookies: Куки, возвращаемые сервером.
    print("Куки, возвращаемые сервером:", response.cookies)
# history: Список объектов Response, представляющих историю перенаправлений.
    print("История перенаправлений:", response.history)
# ok: Логический атрибут, указывающий, был ли запрос успешным (коды 2xx).
    print("Запрос успешен (коды 2xx):", response.ok)
# reason: Сообщение статуса HTTP (например, "OK", "Not Found").
    print("Сообщение статуса HTTP:", response.reason)

### Передача параметров в URL params=

In [ ]:
import requests
from bs4 import BeautifulSoup

# Базовый URL для поиска в онлайн-магазине
base_url = 'https://example.com/search'

# Параметры поиска: ищем ноутбуки с определенными характеристиками
search_params = {
    'query': 'note',  # Поисковый запрос
    'brand': 'Dell',  # Бренд
    'min_price': '50000',  # Минимальная цена
    'max_price': '100000',  # Максимальная цена
    'sort': 'price_asc'  # Сортировка по возрастанию цены
}

# Отправляем GET-запрос с параметрами
response = requests.get(base_url, params=search_params)

# Проверка успешности запроса
if response.status_code == 200:
    # Парсим HTML-страницу с использованием BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Находим все элементы, соответствующие карточкам товаров (здесь это примерный CSS-селектор)
    product_cards = soup.select('.product-card')

    for card in product_cards:
        # Извлекаем информацию из карточки товара (название и цена)
        product_name = card.select_one('.product-name').text
        product_price = card.select_one('.product-price').text

        print(f'Название товара: {product_name}')
        print(f'Цена товара: {product_price}')
        print('---')
else:
    print(f'Не удалось выполнить запрос. Код ошибки: {response.status_code}')
    print(response.url)

### Скачиваем файл по прямой ссылке

In [ ]:
скачиваем файл по прямой ссылке, делим его на куски по 100 000 байт чтобы ускорить процесс и не грузить оперативную память
stream=true поддерживаем соединение открытым
wb - режим бинарных данных

In [ ]:
# сохраняем файл кусочками
import requests
response = requests.get(url='https://parsinger.ru/video_downloads/videoplayback.mp4', stream=True)
with open('file.mp4', 'wb') as video:
    for piece in response.iter_content(chunk_size=100000):
        video.write(piece)

### Скачиваем с сайта все изображения (код из ChatGPT).

In [ ]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
from urllib.request import urlretrieve

# URL сайта
url = "https://parsinger.ru/img_download/index.html"
# Папка для сохранения изображений
download_directory = "images_parsing_lesson"
# Создаем папку для сохранения изображений, если она не существует
if not os.path.exists(download_directory):
    os.makedirs(download_directory)

# Функция для получения всех изображений на странице
def get_all_images(page_url):
    try:
        response = requests.get(page_url)
        response.raise_for_status()  # Проверяем успешность запроса
        soup = BeautifulSoup(response.content, "html.parser")
        image_tags = soup.find_all("img")
        image_urls = [urljoin(page_url, img["src"]) for img in image_tags]
        return image_urls
    except requests.exceptions.RequestException as e:
        print(f"Ошибка при получении изображений: {e}")
        return []

# Функция для скачивания изображений
def download_images(image_urls, download_folder):
    for image_url in image_urls:
        try:
            filename = os.path.join(download_folder, os.path.basename(urlparse(image_url).path))
            urlretrieve(image_url, filename)
            print(f"Скачано: {filename}")
        except Exception as e:
            print(f"Ошибка при скачивании изображения {image_url}: {e}")

# Получаем все изображения на странице
all_images = get_all_images(url)
# Скачиваем изображения
download_images(all_images, download_directory)


### Используем список Proxy

In [ ]:
proxies_list = [
    {'http': 'http://10.10.36.159:8000', 'https': 'https://10.10.36.159:8000'},
    {'http': 'http://10.10.51.205:8000', 'https': 'https://10.10.51.205:8000'},
    {'http': 'http://10.10.79.216:8000', 'https': 'https://10.10.79.216:8000'}]
proxy_pool = cycle(proxies_list)
url = "http://example.org"

# Создание сессии
session = requests.Session()
for i in range(1, 6):  # Попробуем сделать 5 запросов
    proxy = next(proxy_pool)
    session.proxies.update(proxy)  # Обновление прокси для сессии
    try:
        response = session.get(url, timeout=5)  # Используем сессию для выполнения запроса
        print(f"Request {i}: Success!")
    except requests.exceptions.RequestException as e:
        print(f"Request {i}: Failed, switching proxy. {proxy}")

### Работа с html оффлайн

In [ ]:
# анализ скаченного файла html оффлайн
from bs4 import BeautifulSoup
import requests
import lxml
with open('index.html', 'r', encoding='utf-8') as file:
    soup2 = BeautifulSoup(file, 'lxml')
    print("Анализ файла с использованием менеджера контекста:\n", soup2)

In [ ]:
# можно попробовать вместо get поставить head, но вроде быстрее не стало
# вычисляем сумму статусов ответов от 200 сайтов
url = 'https://parsinger.ru/3.3/2/1.html'
summa = 0
with requests.Session() as s:
    for i in range(1, 201):
        url = 'https://parsinger.ru/3.3/2/'+str(i)+'.html'
        response = s.get(url)
        summa+= response.status_code
print("Сумма статусов:", summa)

In [ ]:
# url = f'http://parsinger.ru/task/1/{x}.html'
# ищем первый рабочий сайт из 200
with requests.Session() as s:
    for i in range(1, 201):
        url = 'https://parsinger.ru/3.3/1/'+str(i)+'.html'
        response = s.get(url)
        if response.status_code == 200:
            print(url)
            print(response.text)
            break

In [ ]:
# ищем самое большое изображение по объему
name_img = ['1663231240183817644.jpg', '1663231245165469794.jpg', '1663231252148267596.jpg', '16632460271311817.jpg', '1663260860165832550.jpg',
 '1663260862112644405.jpg', '1663260864114071369.jpg', '1663260869127473152.jpg', '1663260874115452216.jpg', '1663260877136512181.jpg',
 '1663260878140464277.jpg', '1663267600193799276.jpg', '1663267613117130673.jpg', '1663267619197170483.jpg', '1663267626154597739.jpg',
 '1663267648135114690.jpg', '166326765416196421.jpg', '1663267662118079649.jpg', '1663267668165066872.jpg', '1663267878176341940.jpg',
 '166326990115068678.jpg', '1663269922185881885.jpg', '1663269927127433209.jpg', '1663269942143420441.jpg', '1663269946174943071.jpg',
 '1663269964195277579.jpg', '1663269970148058649.jpg', '1663269974197750992.jpg', '166326997917397750.jpg', '1663270039138442380.jpg',
 '1663388012194470737.jpg', '166342371029995280.jpg', '1663423712288242036.jpg', '1663423715255612089.jpg', '1663423720221155166.jpg',
 '1663423722211139858.jpg', '1663423724211218483.jpg', '1663423728215479371.jpg', '1663423729298828299.jpg', '1663423732225964403.jpg',
 '1663424198111663025.jpg', '1663424199157537861.jpg', '1663424200184778832.jpg', '166342420214123494.jpg', '166342420317539591.jpg',
 '1663424204161674559.jpg', '1663424206188873432.jpg', '166342420813193185.jpg', '1663424209187179962.jpg', '1663424212162573102.jpg']
url = 'https://parsinger.ru/3.3/3/img/'
max_size = 0.0
max = name_img[0]
with requests.Session() as s:
    for img in name_img:
        url = 'https://parsinger.ru/3.3/3/img/' + img
        response = s.get(url)
        if int(response.headers.get('Content-Length')) > max_size:
            max = img
            max_size = int(response.headers.get('Content-Length'))
print(max[:-4])

In [ ]:
url = 'https://parsinger.ru/3.3/4/1.html'
first = 1
last = 1
with requests.Session() as s:
    for i in range(1, 101):
        url = 'https://parsinger.ru/3.3/4/' + str(i) + '.html'
        response = s.get(url)
        if response.status_code == 200:
            first = i
            break
    for j in range(i, 101):
        url = 'https://parsinger.ru/3.3/4/' + str(j) + '.html'
        response = s.get(url)
        if response.status_code == 200:
            last = j
print(f"Первая доступная страница: {first}.html")
print(f"Последняя доступная страница: {last}.html")

In [ ]:
# исправляем нечитаемую кодировку
url = 'https://parsinger.ru/3.4/2/index.html'
response = requests.get(url)
response.encoding = 'utf-8'
print("Содержимое ответа:")
print(response.text)

In [ ]:
url = 'https://parsinger.ru/img_download/img/ready/1.png'
with requests.Session() as s:
    for i in range(1, 6):
        url = 'https://parsinger.ru/img_download/img/ready/' + str(i) + '.png'
        response = s.get(url)
        with open('image'+str(i)+'.jpeg', 'wb') as file:
            file.write(response.content)

In [ ]:
response = requests.get(url='https://parsinger.ru/3.4/1/json_weather.json')
weather = response.json()
min_temperature_date = min(weather, key=lambda x: int(x['Температура воздуха'][:-2]))['Дата']
# список weather состоит из словарей, сперва мы находим такой элемент списка, у которого минимальное значение по ключу [Температура],
# а потом из полученного элемента словаря извлекаем значение по ключу [Дата]
#min_temp = min(weather, key=lambda x: x['Температура воздуха'])
print(min_temperature_date)
print(type(weather))

In [ ]:
response = requests.get(url='https://parsinger.ru/3.4/3/dialog.json')
chat = response.json()

def dict_to_list(d):
    result = []
    for key, value in d.items():
        if isinstance(value, dict):
            result.append({key: dict_to_list(value)})
        else:
            result.append({key: value})
    return result

chat_list = dict_to_list(chat)
print(len(chat_list))

In [ ]:
def flatten_dict(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

# Пример входного словаря
tree_dict = {
    'a': {
        'b': {
            'c': 1,
            'd': 2
        },
        'e': 3
    },
    'f': 4
}

# Преобразование словаря в список с одним уровнем вложенности
flat_list = flatten_dict(tree_dict)
print(flat_list)


In [ ]:
from bs4 import BeautifulSoup

# Пример HTML-строки
html_string = """
<!DOCTYPE html>
<html lang="ru">
<head>
    <meta charset="UTF-8">
    <title>Пример карточки товара</title>
</head>
<body>
    <div class="card">
        <img src="image.jpg" alt="Пример изображения товара">
        <h2 class="card-title"> iPhone 15 </h2>
        <p class="card-description">Аппаратной основой Apple iPhone 15 Pro Max стал 3-нанометровый чипсет A17 Pro с 6-ядерным GPU и поддержкой трассировки лучей. </p>
        <p class="card-price">999 999 руб.</p>
        <a href="https://example.com/product-link" class="card-link">Подробнее</a>
    </div>
</body>
</html>
"""
# Создание объекта BeautifulSoup
soup = BeautifulSoup(html_string, 'html.parser')
# Использование .text для извлечения всего текста из div
div_text = soup.find('div').text
print(div_text)

In [ ]:
from bs4 import BeautifulSoup
html_doc = """
<!DOCTYPE html>
<html lang="ru">
<head>
    <meta charset="UTF-8">
</head>
<body>
    <div class="cards">
        <!-- Карточка товара 1 -->
        <div class="card">
            <img src="parsing.png" alt="WEB Парсинг на Python">
            <h2 class="card-title">WEB Парсинг на Python</h2>
            <p class="card-articul">Артикул: 104774</p>
            <p class="card-stock">Наличие: 5 шт.</p>
            <p class="card-price">3500 руб.</p>
            <a href="https://stepik.org/a/104774" class="card-button">Купить</a>
        </div>
        <!-- Карточка товара 2 -->
        <div class="card">
            <img src="async.png" alt="Асинхронный Python">
            <h2 class="card-title">Асинхронный Python</h2>
            <p class="card-articul">Артикул: 170777</p>
            <p class="card-stock">Наличие: 10 шт.</p>
            <p class="card-price">3500 руб.</p>
            <a href="https://stepik.org/a/170777" class="card-button">Купить</a>
        </div>
        <!-- Карточка товара 3 -->
        <div class="card">
            <img src="selenium.PNG" alt="Selenium Python">
            <h2 class="card-title">Selenium Python</h2>
            <p class="card-articul">Артикул: 119495</p>
            <p class="card-stock">Наличие: 5 шт.</p>
            <p class="card-price">1250 руб.</p>
            <a href="https://stepik.org/a/119495" class="card-button">Купить</a>
        </div>
    </div>
</body>
</html>
"""
def main (html_string):
    # Инициализация объекта BeautifulSoup
    soup = BeautifulSoup(html_string, 'html.parser')

    # Поиск всех элементов с классом 'card-articul'
    articuls = soup.find_all(class_='card-articul')
    for art in articuls:
        print(int(art.text[-6:]))
        
    # Извлечение числовых значений артикулов и их суммирование
    sum_articuls = sum([int(art.text[-6:]) for art in articuls])

    print(f"Сумма артикулов: {sum_articuls}")  # Вывод результата
main(html_doc)

In [ ]:
html_doc = """
<!DOCTYPE html>
<html lang="ru">
<head>
    <meta charset="UTF-8">
</head>
<body>
<div class="card">
    <h2>Товар 1</h2>
    <img src="parsing.png" alt="779966">
    <p>Цена: 1000 руб.</p>
    <p>Описание: Отличный товар, изготовлен из качественных материалов.</p>
    <p>Технические характеристики: Размеры: 10x10x10 см, Вес: 1 кг.</p>
    <p>Доступные размеры: S, M, L</p>
    <p>Отзывы: 5 звезд</p>
    <p>Наличие на складе: В наличии</p>
    <p>Информация о доставке: Бесплатно при заказе от 3000 руб.</p>
    <a href="https://stepik.org/a/104774" class="btn">Купить</a>
</div>
<div class="card">
    <h2>Товар 2</h2>
    <img src="async.png" alt="331155">
    <p>Цена: 1500 руб.</p>
    <p>Описание: Превосходный товар, подходит для повседневного использования.</p>
    <p>Технические характеристики: Размеры: 15x15x15 см, Вес: 1.5 кг.</p>
    <p>Доступные размеры: M, L, XL</p>
    <p>Отзывы: 4.5 звезд</p>
    <p>Наличие на складе: В наличии</p>
    <p>Информация о доставке: Бесплатно при заказе от 5000 руб.</p>
    <a href="https://stepik.org/a/170777" class="btn">Купить</a>
</div>
<div class="card">
    <h2>Товар 3</h2>
    <img src="parsing.png" alt="558877">
    <p>Цена: 2000 руб.</p>
    <p>Описание: Удобный товар для дома и офиса.</p>
    <p>Технические характеристики: Размеры: 12x12x12 см, Вес: 1.2 кг.</p>
    <p>Доступные размеры: S, M</p>
    <p>Отзывы: 4.7 звезд</p>
    <p>Наличие на складе: В наличии</p>
    <p>Информация о доставке: Бесплатно при заказе от 3500 руб.</p>
    <a href="https://stepik.org/a/104774" class="btn">Купить</a>
</div>
<div class="card">
    <h2>Товар 4</h2>
    <img src="async.png" alt="449933">
    <p>Цена: 2500 руб.</p>
    <p>Описание: Стильный и практичный товар.</p>
    <p>Технические характеристики: Размеры: 14x14x14 см, Вес: 1.4 кг.</p>
    <p>Доступные размеры: L, XL</p>
    <p>Отзывы: 4.8 звезд</p>
    <p>Наличие на складе: В наличии</p>
    <p>Информация о доставке: Бесплатно при заказе от 4000 руб.</p>
    <a href="https://stepik.org/a/170777" class="btn">Купить</a>
</div>
<div class="card">
    <h2>Товар 5</h2>
    <img src="parsing.png" alt="667711">
    <p>Цена: 2700 руб.</p>
    <p>Описание: Идеальный товар для повседневного использования.</p>
    <p>Технические характеристики: Размеры: 13x13x13 см, Вес: 1.3 кг.</p>
    <p>Доступные размеры: M, L, XL</p>
    <p>Отзывы: 4.9 звезд</p>
    <p>Наличие на складе: В наличии</p>
    <p>Информация о доставке: Бесплатно при заказе от 4500 руб.</p>
    <a href="https://stepik.org/a/104774" class="btn">Купить</a>
</div>
<div class="card">
    <h2>Товар 6</h2>
    <img src="async.png" alt="334455">
    <p>Цена: 3000 руб.</p>
    <p>Описание: Прочный и надежный товар.</p>
    <p>Технические характеристики: Размеры: 16x16x16 см, Вес: 1.6 кг.</p>
    <p>Доступные размеры: S, M</p>
    <p>Отзывы: 5 звезд</p>
    <p>Наличие на складе: В наличии</p>
    <p>Информация о доставке: Бесплатно при заказе от 5000 руб.</p>
    <a href="https://stepik.org/a/170777" class="btn">Купить</a>
</div>
</body>
</html>
"""
from bs4 import BeautifulSoup
def main(html_string):
    # Инициализация объекта BeautifulSoup
    soup = BeautifulSoup(html_string, 'html.parser')

    # Находим все теги img
    img_tags = soup.find_all('img')
    
    # Извлекаем значения атрибута alt и суммируем их
    total_sum = sum([int(img.get('alt')) for img in img_tags])
    print(f"Сумма всех значений в атрибуте alt тега img: {total_sum}")

main(html_doc)

In [ ]:
import requests
from bs4 import BeautifulSoup
url = 'https://parsinger.ru/4.1/1/index4.html'
response = requests.get(url)
html_content = response.content
soup = BeautifulSoup(html_content, 'html.parser')
prices = soup.find_all('p', attrs = {'class' : 'price product_price'})
count = 0
for price in prices:
    count += int(price.text.replace(' ', '')[:-3]) # убрали пробелы и откинула руб
print(count)

In [ ]:
import requests
from bs4 import BeautifulSoup
url = 'https://parsinger.ru/4.1/1/index4.html'
response = requests.get(url)
html_content = response.content
soup = BeautifulSoup(html_content, 'html.parser')
tags_li = soup.find_all('li')
for tag in tags_li:
    print(tag.get('id'))

In [ ]:
import requests
from bs4 import BeautifulSoup
url = 'https://parsinger.ru/4.1/1/index6.html'
response = requests.get(url)
html_content = response.content
soup = BeautifulSoup(html_content, 'html.parser')
sibling = soup.find(id= 'section3').find('p').next_sibling.strip()
abzac = soup.find(id= 'section3') # для понимания что мы ищем
print(abzac)
print(sibling)

In [ ]:
import requests
from bs4 import BeautifulSoup
url = 'https://parsinger.ru/4.1/1/index5.html'
response = requests.get(url)
html_content = response.content
soup = BeautifulSoup(html_content, 'html.parser')
emails = [mail.find('strong').next_sibling.strip() for mail in soup.find_all('div', class_= 'email_field')]
print(emails)

In [ ]:
html ='''<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Онлайн Магазин Книг</title>
</head>
<body>
    <div class="book-card">
        <img src="1.png" alt="Обложка книги 1" class="book-cover book-cover_hard">
        <h2 class="book pages">Название Книги 1</h2>
        <p class="book-author">Автор: Автор 1</p>
        <p class="book-isbn">ISBN: 978-1234567890</p>
        <p class="book-cover-type">Обложка: Твердая</p>
        <p class="count price">Цена: $20.00</p>
        <p class="book-format">Формат: Мягкая обложка</p>
        <p class="count pages">Количество страниц: 300</p>
        <p class="count stock">Количество на складе: 75</p>
        <p class="book-publisher">Издательство: Издательство 1</p>
        <p class="book-publication-date">Дата публикации: 01.01.2023</p>
        <p class="count rating">Рейтинг: 4.5</p>
        <p class="book-genre">Жанр: Роман</p>
        <p class="book-language">Язык: Английский</p>
        <p class="book-availability">Доступность: В наличии</p>
        <p class="book-description">Описание: Краткое описание книги 1.</p>
        <button class="book-purchase-btn">Добавить в корзину</button>
    </div>
    <div class="book-card">
        <img src="2.png" alt="Обложка книги 2" class="book-cover book-cover_hard">
        <h2 class="book pages">Название Книги 2</h2>
        <p class="book-author">Автор: Автор 2</p>
        <p class="book-isbn">ISBN: 978-9876543210</p>
        <p class="book-cover-type">Обложка: Мягкая</p>
        <p class="count price">Цена: $18.50</p>
        <p class="book-format">Формат: Электронная версия (e-book)</p>
        <p class="count pages">Количество страниц: 250</p>
        <p class="count stock">Количество на складе: 119</p>
        <p class="book-publisher">Издательство: Издательство 3</p>
        <p class="book-publication-date">Дата публикации: 20.03.2023</p>
        <p class="count rating">Рейтинг: 4.7</p>
        <p class="book-genre">Жанр: Детская литература</p>
        <p class="book-language">Язык: Французский</p>
        <p class="book-availability">Доступность: В наличии</p>
        <p class="book-description">Описание: Краткое описание книги 2.</p>
         <button class="book-purchase-btn">Добавить в корзину</button>
    </div>
    <div class="book-card">
        <img src="3.png" alt="Обложка книги 3" class="book-cover book-cover_hard">
        <h2 class="book pages">Название Книги 3</h2>
        <p class="book-author">Автор: Автор 3</p>
        <p class="book-isbn">ISBN: 978-0987654321</p>
        <p class="book-cover-type">Обложка: Твердая</p>
        <p class="count price">Цена: $25.00</p>
        <p class="book-format">Формат: Мягкая обложка</p>
        <p class="count pages">Количество страниц: 400</p>
        <p class="count stock">Количество на складе: 216</p>
        <p class="book-publisher">Издательство: Издательство 2</p>
        <p class="book-publication-date">Дата публикации: 15.02.2023</p>
        <p class="count rating">Рейтинг: 4.8</p>
        <p class="book-genre">Жанр: Фантастика</p>
        <p class="book-language">Язык: Русский</p>
        <p class="book-availability">Доступность: В наличии</p>
        <p class="book-description">Описание: Краткое описание книги 3.</p>
        <button class="book-purchase-btn">Добавить в корзину</button>
    </div>
    <div class="book-card">
        <img src="4.png" alt="Обложка книги 4" class="book-cover book-cover_hard">
        <h2 class="book pages">Название Книги 4</h2>
        <p class="book-author">Автор: Автор 4</p>
        <p class="book-isbn">ISBN: 978-5432109876</p>
        <p class="book-cover-type">Обложка: Твердая</p>
        <p class="count price">Цена: $22.00</p>
        <p class="book-format">Формат: Мягкая обложка</p>
        <p class="count pages">Количество страниц: 350</p>
        <p class="count stock">Количество на складе: 17</p>
        <p class="book-publisher">Издательство: Издательство 4</p>
        <p class="book-publication-date">Дата публикации: 10.04.2023</p>
        <p class="count rating">Рейтинг: 4.9</p>
        <p class="book-genre">Жанр: Детектив</p>
        <p class="book-language">Язык: Английский</p>
        <p class="book-availability">Доступность: В наличии</p>
        <p class="book-description">Описание: Краткое описание книги 4.</p>
        <button class="book-purchase-btn">Добавить в корзину</button>
    </div>
    <div class="book-card">
        <img src="5.png" alt="Обложка книги 5" class="book-cover book-cover_hard">
        <h2 class="book pages">Название Книги 5</h2>
        <p class="book-author">Автор: Автор 5</p>
        <p class="book-isbn">ISBN: 978-8765432109</p>
        <p class="book-cover-type">Обложка: Мягкая</p>
        <p class="count price">Цена: $19.50</p>
        <p class="book-format">Формат: Мягкая обложка</p>
        <p class="count pages">Количество страниц: 280</p>
        <p class="count stock">Количество на складе: 63</p>
        <p class="book-publisher">Издательство: Издательство 5</p>
        <p class="book-publication-date">Дата публикации: 05.05.2023</p>
        <p class="count rating">Рейтинг: 4.6</p>
        <p class="book-genre">Жанр: Фэнтези</p>
        <p class="book-language">Язык: Испанский</p>
        <p class="book-availability">Доступность: В наличии</p>
        <p class="book-description">Описание: Краткое описание книги 5.</p>
        <button class="book-purchase-btn">Добавить в корзину</button>
    </div>
</body>
</html>
'''
from bs4 import BeautifulSoup
def calculate_total_price(html: str) -> float:
    soup = BeautifulSoup(html, 'html.parser')
    price_list = [float(price.text[7:]) for price in soup.find_all('p', class_= "count price")]
    amount_list = [float(q.text[21:]) for q in soup.find_all('p', class_= "count stock")]
    price_amount = list(zip(price_list, amount_list))
    calculate_total_price = sum(a*q for a, q in price_amount)
    return calculate_total_price

print(f"Общая стоимость в случае продажи всех товаров: ${calculate_total_price(html):.2f}")

#### Находим на сайте поля с стоимостью и количеством, перемножаем, узнаем суммарную стоимость всех позиций

In [ ]:
# тоже самое, но короче
calculate_total_price = sum(float(price.text[7:]) * float(q.text[21:])
    for price, q in zip(soup.find_all('p', class_="count price"), soup.find_all('p', class_="count stock")))
print(calculate_total_price)

#### Сравниваем время выполнения find_all для 1 и 2 запросов к сайту

In [ ]:
import requests
from bs4 import BeautifulSoup
import timeit

url = 'https://parsinger.ru/4.3/5/index.html'
response = requests.get(url)
response.encoding= 'utf-8' # Исправление ошибки кодировки
response.raise_for_status()  # Проверка, что запрос был успешным
soup = BeautifulSoup(response.text, 'html.parser')

# Код с двумя вызовами find_all
code_two_calls = '''
prices = [float(price.text[7:]) for price in soup.find_all('p', class_="count price")]
amounts = [float(amount.text[21:]) for amount in soup.find_all('p', class_="count stock")]
calculate_total_price = sum(p * a for p, a in zip(prices, amounts))
'''
# Код с одним вызовом find_all и разделением элементов
code_one_call = '''
elements = soup.find_all('p', class_=["count price", "count stock"])
prices = [float(el.text[12:]) for el in elements if "price" in el['class']]
amounts = [float(el.text[21:]) for el in elements if "stock" in el['class']]
calculate_total_price = sum(p * a for p, a in zip(prices, amounts))
'''
# Измерение времени выполнения
time_two_calls = timeit.timeit(code_two_calls, globals=globals(), number=1000)
time_one_call = timeit.timeit(code_one_call, globals=globals(), number=1000)

print(f'Время выполнения с двумя вызовами find_all: {time_two_calls:.6f} секунд')
print(f'Время выполнения с одним вызовом find_all: {time_one_call:.6f} секунд')

#### Находоим на странице цены на все товары и складываем общую стоимость

In [ ]:
url = 'https://parsinger.ru/html/index1_page_1.html'
import requests
from bs4 import BeautifulSoup
response = requests.get(url)
html_content = response.content
soup = BeautifulSoup(html_content, 'html.parser')
prices = sum(int(p.text[:-4]) for p in soup.find_all('p', class_= 'price'))
print(prices)


In [ ]:
url = 'https://parsinger.ru/html/hdd/4/4_1.html'
import requests
from bs4 import BeautifulSoup
response = requests.get(url)
html_content = response.content
soup = BeautifulSoup(html_content, 'html.parser')
old_price = int(soup.find(id = 'old_price').text[:-4])
new_price = int(soup.find(id = 'price').text[:-4])
discount = (old_price - new_price)*100/old_price
print(f"{discount:.1f}")

#### с 4 страниц сайта собираем названия товаров и складываем в двумерный список

In [ ]:
url = 'https://parsinger.ru/html/index3_page_'
import requests
from bs4 import BeautifulSoup
names_table = []
for i in range(1, 5):
    site = url + str(i) + '.html'
    response = requests.get(site)
    html_content = response.content
    soup = BeautifulSoup(html_content, 'html.parser')
    names_page = [name.text for name in soup.find_all('a', class_= 'name_item')]
    names_table.append(names_page)
print(names_table)    

In [ ]:
url = 'https://parsinger.ru/html/mouse/3/3_'
import requests
from bs4 import BeautifulSoup
art_sum = 0
for i in range(1, 33):
    site = url + str(i) + '.html'
    response = requests.get(site)
    html_content = response.content
    soup = BeautifulSoup(html_content, 'html.parser')
    art_sum+= int(soup.find('p', class_= 'article').text[9:])
print(art_sum)    

In [ ]:
url0 = 'https://parsinger.ru/html/'
#https://parsinger.ru/html/watch/1/1_1.html
#https://parsinger.ru/html/mobile/2/2_1.html
#https://parsinger.ru/html/mouse/3/3_1.html
#https://parsinger.ru/html/hdd/4/4_1.html
#https://parsinger.ru/html/headphones/5/5_1.html

page_names = ['watch/1/1_', 'mobile/2/2_', 'mouse/3/3_', 'hdd/4/4_', 'headphones/5/5_']
amount_on_page = [33, 33, 33, 33, 33]
summa = 0
for i in range(5):
    for j in range(1, amount_on_page[i]):
        site = url0 + page_names[i] + str(j) + '.html'
        response = requests.get(site)
        html_content = response.content
        soup = BeautifulSoup(html_content, 'html.parser')
        price = int(soup.find(id = 'price').text[:-4])
        amount = int(soup.find(id = 'in_stock').text[11:])
        summa+= price*amount
print(summa)

In [ ]:
import requests
from bs4 import BeautifulSoup

url = 'http://stepik-parsing.ru/html/index1_page_1.html'
response = requests.get(url=url)
response.encoding = 'utf-8'
soup = BeautifulSoup(response.text, 'lxml')

pagen = int(soup.find('div', 'pagen').find_all('a')[-1].text)
category = [f"http://stepik-parsing.ru/html/{x['href']}" for x in soup.find('div', 'nav_menu').find_all('a')]

results = []
for x in range(1, len(category)+1):
    for i in range(1, pagen+1):
        url = f'http://stepik-parsing.ru/html/index{x}_page_{i}.html'
        response = requests.get(url=url)
        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.text, 'html.parser')
        name = [x.text.strip() for x in soup.find_all('a', 'name_item')]
        link = [f"http://stepik-parsing.ru/html/{x['href'].strip()}" for x in soup.find_all('a', class_='name_item')]
        result = [int(x.text.strip().split()[0]) for x in soup.find_all('p', 'price')]

        for li, res in zip(link, result):
            response = requests.get(url=li)
            response.encoding = 'utf-8'
            soup = BeautifulSoup(response.text, 'lxml')
            count = int(soup.find('span', id='in_stock').string.split(':')[1])
            results.append(res * count)
print(sum(results))

In [ ]:
import requests
url = 'https://cbr.ru/Queries/AjaxDataSource/112805?DT=&val_id=R01239&_=1669744293513'
response_euro = requests.get(url=url).json()
#headers = {'user-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:106.0) Gecko/20100101 Firefox/106.0',
#           'X-Requested-With': 'XMLHttpRequest', }
#url = 'https://cbr.ru/Queries/AjaxDataSource/112805'
data_dollar = {
    'DT': '',
    'val_id': 'R01235',
    '_': '1667219511852'}
data_euro = {
    'DT': '',
    'val_id': 'R01239',
    '_': '1667219511853'}
#response_dollar = requests.get(url=url, headers=headers, params=data_dollar).json()[-1]
#response_euro = requests.get(url=url, headers=headers, params=data_euro).json()[-1]

#print(f'Дата: {response_dollar["data"][:10]}')
#print(f'Курс USD: {response_dollar["curs"]} рублей')
print(f'Курс EUR: {response_euro["curs"]} рублей')

In [ ]:
import requests

url_usd = 'https://cbr.ru/cursonweek/?DT=&val_id=R01235&_=1692528097005'
url_eur = 'https://cbr.ru/cursonweek/?DT=&val_id=R01239&_=1692528097006'
url_cny = 'https://cbr.ru/cursonweek/?DT=&val_id=R01375&_=1692528097007'

urls_list = [url_usd, url_eur, url_cny]
ex_list = ['USD', 'EUR', 'CNY']

for i, j in zip(urls_list, ex_list):
    response = requests.get(i).json()[-1]
    print(f"Курс {j} на {response['data'][:-9]} к рублю - {response['curs']}")

In [12]:
# складываем значения элементов из первой колонки таблицы
url = 'https://parsinger.ru/table/2/index.html'
import requests
from bs4 import BeautifulSoup
response = requests.get(url)
html_content = response.content
soup = BeautifulSoup(html_content, 'html.parser')
first_colomn = [float(column.find_all('td')[0].text) for column in soup.find_all('tr')[1:]]
summa = sum(first_colomn)
print(summa)

80.726


In [18]:
# в таблице в каждой строчке есть 1 оранжевая ячейка и 1 синяя (она всегда последняя), требуется перемножить значения и суммировать
url = 'https://parsinger.ru/table/5/index.html'
import requests
from bs4 import BeautifulSoup
response = requests.get(url)
html_content = response.content
soup = BeautifulSoup(html_content, 'html.parser')
orange_list = [[float(row.find(class_ = 'orange').text), int(row.find_all('td')[-1].text)] for row in soup.find_all('tr')[1:]]
summa = sum([elem[0]*elem[1] for elem in orange_list])
print(summa)

2521465.6860000016


In [ ]:
# подбираем машину по критериям
url = 'https://parsinger.ru/4.8/6/index.html'
import requests, json
from bs4 import BeautifulSoup
response = requests.get(url)
html_content = response.content
soup = BeautifulSoup(html_content, 'html.parser')
#columns_names = [name.text for name in soup.find_all('th')]
indices = [0, 1, 4, 7]
columns_names = [soup.find_all('th')[i].text for i in indices]
filtered_cars = []
rows = soup.find_all('tr')[1:]
for row in rows:
    row_data =  dict(zip(columns_names, (row.find_all('td')[j].text for j in indices)))
    if row_data['Тип двигателя'] == 'Бензиновый' and int(row_data['Стоимость авто']) <= 4000000 and int(row_data['Год выпуска']) >= 2005:
        filtered_cars.append(row_data)
for car in filtered_cars:
    car["Стоимость авто"] = int(car["Стоимость авто"])
    car['Год выпуска'] = int(car['Год выпуска'])
filtered_cars.sort(key=lambda x: x["Стоимость авто"])
sorted_cars = json.dumps(filtered_cars, indent=4, ensure_ascii=False)
#sorted(sorted_cars, key=lambda x: x["Стоимость авто"])
print(sorted_cars)

In [81]:
# собираем информацию про жесткие диски с 4 страниц
import csv
import requests
from bs4 import BeautifulSoup

with open('result_stepik.csv', 'w', encoding='utf-8-sig', newline='') as file:
    writer = csv.writer(file, delimiter=';')
    writer.writerow(['Наименование', 'Бренд', 'Форм-фактор', 'Ёмкость', 'Объем буферной памяти', 'Цена'])
#url = 'https://parsinger.ru/html/index4_page_1.html'
for page in range(1, 5):
    url = f'https://parsinger.ru/html/index4_page_{page}.html'
    response = requests.get(url=url)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'lxml')
    name = [x.text.strip() for x in soup.find_all('a', class_='name_item')]
    description = [x.text.split('\n') for x in soup.find_all('div', class_='description')]
    price = [x.text for x in soup.find_all('p', class_='price')]
    with open('result_stepik.csv', 'a', encoding='utf-8-sig', newline='') as file:
        writer = csv.writer(file, delimiter=';')
        for item, descr, price in zip(name, description, price):
            # Формируем строку для записи
            flatten = item, *[x.split(':')[1].strip() for x in descr if x], price
            writer.writerow(flatten)
print('Файл result_stepik.csv создан')

Файл result_stepik.csv создан


In [149]:
import requests, csv
from bs4 import BeautifulSoup

with open('watches_stepik.csv', 'w', encoding='utf-8-sig', newline='') as file:
   writer = csv.writer(file, delimiter=';')
   writer.writerow(['Наименование', 'Артикул', 'Бренд', 'Модель', 'Тип', 'Технология экрана',
                    'Материал корпуса', 'Материал браслета', 'Размер', 'Сайт производителя',
                    'Наличие', 'Цена', 'Старая цена', 'Ссылка на карточку с товаром'])

# сперва в 'pagen' находим адреса 4 страниц с перечнем часов
url = 'https://parsinger.ru/html/index1_page_1.html'
response = requests.get(url)
html_content = response.content
soup = BeautifulSoup(html_content, 'html.parser')
sites = ['https://parsinger.ru/html/' + big_page.get('href') for big_page in soup.find('div', class_='pagen').find_all('a')]

# теперь с этих 4 страниц собираем адреса персональных страниц каждых часов
all_pages = []
for page in sites:
    response = requests.get(page)
    html_content = response.content
    soup = BeautifulSoup(html_content, 'html.parser')
    current_page = [i.find('a').get('href') for i in soup.find_all('div', class_ = 'img_box')]
    all_pages.extend(current_page)

# теперь со всех персональных страниц собираем всю нужную информацию
for page1 in all_pages:
    site1 = 'https://parsinger.ru/html/' + page1
    response = requests.get(site1)
    html_content = response.content
    soup1 = BeautifulSoup(html_content, 'html.parser')
    name = [soup1.find(id = 'p_header').text, soup1.find('p', class_ = 'article').text.split(':')[1].strip()]
    description = [x.text.split(':')[1].strip() for x in soup1.find_all('li')]
    last = [soup1.find(id = 'in_stock').text.split(':')[1].strip(), soup1.find('span', id = 'price').text, soup1.find('span', id = 'old_price').text]
    flatten = *name, *description, *last, site1
    # print(flatten)
    with open('watches_stepik.csv', 'a', encoding='utf-8-sig', newline='') as file:
        writer = csv.writer(file, delimiter=';')
        writer.writerow(flatten)
print('Файл watches_stepik.csv создан') 

Файл watches_stepik.csv создан


In [155]:
import requests
from bs4 import BeautifulSoup
import json

url = 'https://parsinger.ru/html/index4_page_1.html'
response = requests.get(url)
response.encoding = 'utf-8'
soup = BeautifulSoup(response.text, 'lxml')
sites = ['https://parsinger.ru/html/' + big_page.get('href') for big_page in soup.find('div', class_='pagen').find_all('a')]

result_json = []
for page in sites:
    response = requests.get(page)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'lxml')
    name = [x.text.strip() for x in soup.find_all('a', class_='name_item')]
    description = [x.text.strip().split('\n') for x in soup.find_all('div', class_='description')]
    price = [x.text for x in soup.find_all('p', class_='price')]
    for list_item, price_item, name in zip(description, price, name):
        result_json.append({
            'Наименование': name,
            'Бренд': [x.split(':')[1].strip() for x in list_item][0],
            'Форм-фактор': [x.split(':')[1].strip() for x in list_item][1],
            'Ёмкость': [x.split(':')[1].strip() for x in list_item][2],
            'Объем буферной памяти': [x.split(':')[1].strip() for x in list_item][3],
            'Цена': price_item
        })

with open('hdd.json', 'w', encoding='utf-8') as file:
    json.dump(result_json, file, indent=4, ensure_ascii=False)
print('Файл hdd.json создан')

Файл hdd.json создан


In [157]:
from telethon import TelegramClient, events, sync, connection

ModuleNotFoundError: No module named 'telethon'